# Running a Qiskit QAOA circuit with Quasar/Vulcan

In this notebook, we will show how to take a QAOA circuit written by Qiskit, transform it into a Quasar circuit with *qusetta*, and then run it with various Quasar features. If you want to install *qusetta* in a virtual enviroment and then add your virtual enviroment to your list of jupyter kernels, see [this post](https://janakiev.com/blog/jupyter-virtual-envs/) for more details.

**Contents**
1. [The problem](#1.-The-problem)
2. [The circuits](#2.-The-circuits)
3. [The cost function](#3.-The-cost-function)
4. [Getting the cover](#4.-Getting-the-cover)
5. [Simulating the circuits](#5.-Simulating-the-circuits)
6. [Optimizing the QAOA parameters](#6.-Optimizing-the-QAOA-parameters)
7. [Outlook](#7.-Outlook)
---

In [ ]:
!pip install qiskit-aer


In [ ]:
 !pip install git+https://github.com/qcware/qusetta

In [ ]:
# install tqdm for progress bars
!pip install tqdm
from tqdm.notebook import tnrange, tqdm

## 1. The problem

For this example, we will work with the Vertex Cover problem. We will follow the procedure in this [qiskit example notebook](https://github.com/Qiskit/qiskit-community-tutorials/blob/master/optimization/vertex_cover.ipynb) to create the problem and the QAOA circuit in qiskit.

We'll begin exactly as they begin -- by creating a random graph (and seeding random for reproducability of the notebook).

In [ ]:
import numpy as np
#!pip install qiskit==0.19.0
#!pip install qiskit-aqua
from qiskit.optimization.applications.ising.common import random_graph

np.random.seed(123)
num_nodes = 6
w = random_graph(num_nodes, edge_prob=0.8, weight_range=10)

Next we'll create the qubit operator and the corresponding QAOA instance. We'll fix a depth $p$ to work with throughout this notebook.

In [ ]:
from qiskit.optimization.applications.ising import vertex_cover
from qiskit.aqua.algorithms import QAOA

qubit_op, offset = vertex_cover.get_operator(w)

p = 4
qaoa = QAOA(qubit_op, p=p)

## 2. The circuits

Now we'll write a function that takes in $\beta_1, \dots, \beta_p$, and $\gamma_1, \dots, \gamma_p$ and outputs the corrresponding qiskit QAOA circuit. Note that `params` is a list such that `params[:p]` is $[\gamma_1, \dots, \gamma_p]$ and `params[p:]` is $[\beta_1, \dots, \beta_p]$.

In [ ]:
import qiskit
from typing import List

def create_qiskit_circuit(params: List[float]) -> qiskit.QuantumCircuit:
    assert len(params) == 2 * p, "invalid number of angles"
    return qaoa.var_form.construct_circuit(params)

Next we'll write a function that uses *qusetta* to convert the qiskit circuit to a quasar circuit.

In [ ]:
#!pip install git+https://github.com/qcware/qusetta
    

In [ ]:
import qusetta as qs
import quasar
# import vulcan

def create_quasar_circuit(params: List[float]) -> quasar.Circuit:
    qiskit_circuit = create_qiskit_circuit(params)
    return qs.Qiskit.to_quasar(qiskit_circuit)

## 3. The cost function

The cost function of the circuit is the expectation value of the qubit operator.

In [ ]:
def expectation_value(statevector: np.ndarray) -> float:
    # note that the second element (eg [1]) is the standard deviation
    return offset + qubit_op.evaluate_with_statevector(statevector)[0].real

## 4. Getting the cover

We can get the Vertex Cover from the statevector outputted by the circuit. We'll choose the cover that we have the highest probability of sampling from the statevector as is done in qiskit's original notebook.

In [ ]:
from qiskit.optimization.applications.ising.common import sample_most_likely

def get_cover(statevector: np.ndarray) -> List[int]:
    return [
        int(x) for x in
        vertex_cover.get_graph_solution(sample_most_likely(statevector))
    ]

# Benchmarking #

Let's generate a number of circuits in increasing size using Qiskit, convert them to Quasar, and then time evaluations of those circuits with some expectation value operators applied to the statevectors.

# Generation of Circuits in Qiskit #

Let's create a list of circuits:

In [ ]:
import time
import numpy as np
from qiskit.optimization.applications.ising.common import random_graph
from qiskit.optimization.applications.ising import vertex_cover
from qiskit.aqua.algorithms import QAOA
from qcware.circuits.quasar_backend import QuasarBackend

Max_circuit_width = 20
# Our circuit widths in number of qubits
Circuit_widths = list(range(4, Max_circuit_width+1, 2))
# right now hardcoding the parameters at 4, which is a little ridiculous as they should 
# expand as we increase qubits
Num_parameters = [1]*len(Circuit_widths)
Parameters = [list(np.random.random(2*x)*np.pi) for x in Num_parameters]
# print(Parameters)

def create_qiskit_circuit(num_nodes: int, params: List[float], edge_prob=0.8, weight_range=10)->qiskit.QuantumCircuit:
    # print(locals())
    np.random.seed(123)
    w = random_graph(num_nodes, edge_prob=edge_prob, weight_range=weight_range)
    qubit_op, offset = vertex_cover.get_operator(w)
    qaoa = QAOA(qubit_op, p=int(len(params)/2))
    return qaoa.var_form.construct_circuit(params)

# Building the Circuits #

Now let's build the circuits; this can take a couple of minutes

In [ ]:
qiskit_circuits = [create_qiskit_circuit(width, parameters) for (width, parameters) in tqdm(list(zip(Circuit_widths, Parameters)), desc="Creating Qiskit circuits")]

## Conversion to Quasar ##

We'll also make a list of quasar circuits converted from these qiskit circuits using `qusetta`.  As you can see, conversion is extremely fast:

In [ ]:
quasar_circuits = [qs.Qiskit.to_quasar(circuit) for circuit in tqdm(qiskit_circuits, desc="Converting Qiskit circuits to Quasar")]

## Pauli expectation values ##

For each circuit we're going to evaluate the statevector and then calculate some expectation value on the result.  These expectation values will be calculated by the creation of a Pauli operator using the `quasar` toolkit.

In [ ]:
I,X,Y,Z = quasar.Pauli.IXYZ()

def x_expectation_pauli(num_qubits: int)->quasar.Pauli:
    result = quasar.Pauli()
    for i in range(num_qubits):
        result += X[i]
    return result

#p = x_expectation_pauli(3)
#print(p)
#print(p.to_matrix())
paulis = [x_expectation_pauli(x) for x in tqdm(Circuit_widths, desc="Generating Paulis")]

# Timing the implementations #

We'll build just a simple decorator to take the clock time before and after a function so we can time how long it takes to evaluate the circuits we've built and apply some expectation value operator to them.  This isn't a sophisticated benchmark, but should give us a good idea of the speedup to expect.  It simply takes a function and decorates it so that it returns a tuple of `(time in evaluation, function result)`

In [ ]:
from functools import wraps
import time
def timed_function(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = f(*args, **kwargs)
        end_time = time.time()
        time_taken = end_time - start_time
        return (time_taken, result)
    return wrapper
                
# to demonstrate
@timed_function
def simple_demo():
    return 42*42
                
print(simple_demo())

# Timing the Qiskit engine #

Now let's get timing information for our circuits by timing the evaluation in Qiskit:

In [ ]:
@timed_function
def eval_qiskit(circuit: qiskit.QuantumCircuit, pauli: quasar.Pauli):
    backend = qiskit.BasicAer.get_backend('statevector_simulator')
    statevector = qiskit.execute(circuit, backend).result().get_statevector()
    # print(statevector)
    # do magical statevector expectation operator pauli stuff here
    # op = pauli.to_matrix()
    # return statevector * op * statevector.T
    # that runs out of memory real fast obviously so use the sparser
    # Pauli matrix_vector_product
    return np.sum(statevector.conj() * pauli.matrix_vector_product(statevector))
    
#print(qiskit_circuits[0])
#eval_qiskit(qiskit_circuits[0], paulis[0])
qiskit_results = [eval_qiskit(circuit, pauli) for (circuit, pauli) in tqdm(list(zip(qiskit_circuits, paulis)), desc="Evaluating qiskit circuits")]
qiskit_times = [x[0] for x in qiskit_results]
qiskit_expectation_values = [x[1] for x in qiskit_results]

# Timing Vulcan #

We can use the QCWare Vulcan engine to evaluate the same circuits in a fraction of the time:

In [ ]:
import qcware
from qcware.circuits.quasar_backend import QuasarBackend
qcware.config.set_host("https://api.hammer.qcware.com")
qcware.config.set_api_key("QCWARE")

@timed_function
def eval_vulcan(circuit: quasar.Circuit, pauli: quasar.Pauli):
    backend = QuasarBackend('vulcan/simulator')
    #sv = backend.run_statevector(circuit=circuit)
    #print(sv)
    result = backend.run_pauli_expectation_value(circuit=circuit, pauli=pauli)
    return result
  
#print(quasar_circuits[0])
#eval_vulcan(quasar_circuits[0], paulis[0])
vulcan_results = [eval_vulcan(circuit, pauli) for (circuit, pauli) in tqdm(list(zip(quasar_circuits, paulis)), desc="Evaluating circuits with vulcan")]
vulcan_times = [x[0] for x in vulcan_results]
vulcan_expectation_values = [x[1] for x in vulcan_results]

In [ ]:
print("Are the results similar?")
print(np.allclose(qiskit_expectation_values, vulcan_expectation_values))

Note that sometimes even very small numerical differences between the simulators can have an effect on the optimization routine (this often has a bigger effect as $p$ gets larger), so the results may be different.

## 7. Outlook

The quasar simulator slightly outperformed the qiskit simulator. But quasar's Vulcan simulator *significantly* speeds up the optimization process.

In [ ]:
import matplotlib.pyplot as plt
xs = Circuit_widths
plt.figure(figsize=(10,7))
plt.plot(xs, qiskit_times, 'ro-', label="qiskit" )
plt.plot(xs, vulcan_times, 'go-', label="vulcan forge measurement")
plt.xlabel('number of qubits')
plt.ylabel('seconds for evaluation')
plt.legend()
ax2 = plt.gca().twinx()
ax2.set_ylabel("speedup")
ax2.plot(xs, np.array(qiskit_times)/np.array(vulcan_times), 'bo-', label='speedup')
ax2.legend(loc='upper center')
plt.show()

[Back to top](#Running-a-Qiskit-QAOA-circuit-with-Quasar/Vulcan).